In [5]:

import osmnx as ox
import numpy as np
import networkx as nx
import folium
import math

# 自宅周辺の軽いデータ
latitude = 35.336452
longitude = 136.922696
dist = 5000
# キャッシュを使う
ox.config(use_cache=True, log_console=True)

graph = ox.graph_from_point(center_point=(latitude,longitude),
                            network_type='drive',
                           simplify=True,
                           retain_all=True,
                           dist=dist,
                           custom_filter='["highway"~"secondary|secondary_link|primary|primary_link|trunk|trunk_link"]["lanes"=2]')
graph2 = ox.graph_from_point(center_point=(latitude,longitude),
                            network_type='drive',
                           simplify=True,
                           retain_all=True,
                           dist=dist,
                           custom_filter='["highway"~"tertiary"]')
graph = nx.compose(graph, graph2)

gdf_edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)
gdf_nodes = ox.graph_to_gdfs(graph, nodes=True, edges=False)

gdf_edges = gdf_edges[gdf_edges["length"] >= 800]

# 逆方向の道を削除する
drop_target = []
for index, row in gdf_edges.iterrows():
    # drop_indexにindex[1], index[0]が存在する場合はなにもしない
    if (index[1], index[0], 0) in drop_target:
        continue
    if (index[0], index[1], 0) in drop_target:
        continue
    drop_target.append(index)
gdf_edges = gdf_edges[gdf_edges.index.isin(drop_target)]

def calculate_angle_between_vectors(A, B, C):
    vector_AB = np.array(B) - np.array(A)
    vector_BC = np.array(C) - np.array(B)
    
    dot_product = np.dot(vector_AB, vector_BC)
    norm_AB = np.linalg.norm(vector_AB)
    norm_BC = np.linalg.norm(vector_BC)
    
    cosine_theta = dot_product / (norm_AB * norm_BC)
    angle_rad = np.arccos(cosine_theta)
    angle_deg = np.degrees(angle_rad)
    return angle_deg

# 座標間の角度の変化の合計値を求める
gdf_edges['geometory_angle_total'] = gdf_edges['geometry'].apply(
    lambda x: sum([calculate_angle_between_vectors(x.coords[i-1], x.coords[i], x.coords[i+1]) for i in range(1, len(x.coords)-1)])
)

# 合計角度と距離の比率を求める
gdf_edges['geometory_angle_rate'] = 1 - gdf_edges['geometory_angle_total'] / gdf_edges['length']

# エッジ内のノード数を求める
graph_all = ox.graph_from_point(center_point=(latitude,longitude),
                            network_type='drive',
                           simplify=True,
                           retain_all=True,
                           dist=dist)
all_nodes = ox.graph_to_gdfs(graph_all, nodes=True, edges=False)
gdf_edges['branch_cnt'] = 0
for index, row in gdf_edges.iterrows():
  # ジオメトリーの座標と一致するノードを取得する
  nodes = all_nodes[all_nodes.geometry.intersects(row.geometry)]
  # 進行方向と逆方向のノードを除外して分岐数を計算する
  gdf_edges.at[index, 'branch_cnt'] = nodes['street_count'].sum() - (len(nodes) * 2)
gdf_edges['branch_cnt_rate'] = 1 - gdf_edges['branch_cnt'] / gdf_edges['length']

# 評価する
gdf_edges['score'] = gdf_edges['geometory_angle_rate'] * gdf_edges['branch_cnt_rate']
gdf_edges['score_normalized'] = gdf_edges['score'] / gdf_edges['score'].max()
gdf_edges['rank'] = gdf_edges['score_normalized'].rank(ascending=False)
# score_normalizedを並び替え
gdf_edges = gdf_edges.sort_values('score_normalized', ascending=False)

# google_map_urlを作成
gdf_edges['google_map_url'] = gdf_edges['geometry'].apply(
    lambda x: f"https://www.google.com/maps/dir/{x.coords[0][1]},{x.coords[0][0]}/'{x.coords[-1][1]},{x.coords[-1][0]}'"
)
# street_view_urlを作成
gdf_edges['street_view_url'] = gdf_edges['geometry'].apply(
    lambda x: f"https://www.google.com/maps/@{x.coords[0][1]},{x.coords[0][0]},20?layer=c&cbll={x.coords[-1][1]},{x.coords[-1][0]}&cbp=12,0,0,0,0"
)

# street_view_urlを作成
gdf_edges['street_view_url'] = gdf_edges['geometry'].apply(
    lambda x: f"https://www.google.com/maps/@{x.coords[0][1]},{x.coords[0][0]},20?layer=c&cbll={x.coords[-1][1]},{x.coords[-1][0]}&cbp=12,0,0,0,0"
)

# googe_earth_urlを作成
def generate_google_earth_url(row):
    # row.geometry配列の中央の値を抽出する
    center_index = math.floor(len(row.geometry.coords) / 2) - 1
    center = row.geometry.coords[center_index]
    return f"https://earth.google.com/web/search/{center[1]},+{center[0]}"

gdf_edges['google_earth_url'] = gdf_edges.apply(generate_google_earth_url, axis=1)

graph = ox.graph_from_gdfs(gdf_nodes, gdf_edges)

# 地図を表示する
map_osm = ox.plot_graph_folium(graph, edge_width=2)

# 候補の上位10件を表示
map_osm.add_child(
    folium.features.GeoJson(
        gdf_edges.head(10).to_json(),
        # https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html
        style_function=lambda x: {
            'color': "#FF0000",
            'weight': 7,
            'opacity': (1 - (x['properties']['rank'] - 1) * 0.09)
        }
    )
    # 緯度と経度を表示する
    .add_child(folium.features.GeoJsonPopup(
        fields=['geometory_angle_total', 'length', 'branch_cnt', 'score_normalized', 'google_map_url', 'street_view_url', 'google_earth_url'],
        aliases=['geometory_angle_total', 'length', 'branch_cnt', 'score_normalized', 'google_map_url', 'street_view_url', 'google_earth_url'],
        localize=True
    ))
)

/tmp/ipykernel_27433/1977741456.py:12: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)
/tmp/ipykernel_27433/1977741456.py:118: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  map_osm = ox.plot_graph_folium(graph, edge_width=2)
